In [1]:
import pandas as pd
import requests
import time

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
#TICKERS_URL = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
top_30_tickers = [
    "AAPL", "MSFT", "NVDA", "AMZN", "META", "BRK.B", "GOOGL", "AVGO", "TSLA", "GOOG",
    "LLY", "JPM", "V", "XOM", "NFLX", "COST", "UNH", "JNJ", "PG", "MA",
    "CVX", "MRK", "PEP", "ABBV", "ADBE", "WMT", "BAC", "HD", "KO", "TMO"
]
LIMIT = 10  # number of past reports (1 = latest)
SLEEP_TIME = 1  # avoid rate limit

# === LOAD TICKERS ===
print("Loading S&P 500 tickers...")
tickers = top_30_tickers

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit={LIMIT}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return []

# === MAIN LOOP ===
all_data = []

print("Fetching data for each ticker...")
for i, symbol in enumerate(tickers):
    print(f"[{i+1}/{len(tickers)}] {symbol}")

    income_list = get_fmp_data("income-statement", symbol)
    balance_list = get_fmp_data("balance-sheet-statement", symbol)
    cashflow_list = get_fmp_data("cash-flow-statement", symbol)

    for idx in range(LIMIT):
        income = income_list[idx] if idx < len(income_list) else {}
        balance = balance_list[idx] if idx < len(balance_list) else {}
        cashflow = cashflow_list[idx] if idx < len(cashflow_list) else {}

        if income or balance or cashflow:
            merged = {
                "symbol": symbol,
                "reportIndex": idx + 1,
                "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
                **income,
                **{f"balance_{k}": v for k, v in balance.items()},
                **{f"cashflow_{k}": v for k, v in cashflow.items()}
            }
            all_data.append(merged)

    time.sleep(SLEEP_TIME)

# === SAVE RESULTS ===
df_company_fondamentals = pd.DataFrame(all_data)
df_company_fondamentals.to_parquet("df_company_fundamentals.parquet", index=False)
print("✅ Data saved to 'df_company_fundamentals.parquet'")


Loading S&P 500 tickers...
Fetching data for each ticker...
[1/30] AAPL
[2/30] MSFT
[3/30] NVDA
[4/30] AMZN
[5/30] META
[6/30] BRK.B
[7/30] GOOGL
[8/30] AVGO
[9/30] TSLA
[10/30] GOOG
[11/30] LLY
[12/30] JPM
[13/30] V
[14/30] XOM
[15/30] NFLX
[16/30] COST
[17/30] UNH
[18/30] JNJ
[19/30] PG
[20/30] MA
[21/30] CVX
[22/30] MRK
[23/30] PEP
[24/30] ABBV
[25/30] ADBE
[26/30] WMT
[27/30] BAC
[28/30] HD
[29/30] KO
[30/30] TMO
✅ Data saved to 'df_company_fundamentals.parquet'


In [2]:
df_company_fondamentals.shape

(150, 134)

In [3]:
df_company_fondamentals.head(15)

,symbol,reportIndex,reportDate,date,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,...,cashflow_netCashUsedProvidedByFinancingActivities,cashflow_effectOfForexChangesOnCash,cashflow_netChangeInCash,cashflow_cashAtEndOfPeriod,cashflow_cashAtBeginningOfPeriod,cashflow_operatingCashFlow,cashflow_capitalExpenditure,cashflow_freeCashFlow,cashflow_link,cashflow_finalLink
0,AAPL,1,2024-09-28,2024-09-28,USD,0000320193,2024-11-01,2024-11-01 06:01:36,2024,FY,...,-121983000000,0,-794000000,29943000000,30737000000,118254000000,-9447000000,108807000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
1,AAPL,2,2023-09-30,2023-09-30,USD,0000320193,2023-11-03,2023-11-02 18:08:27,2023,FY,...,-108488000000,0,5760000000,30737000000,24977000000,110543000000,-10959000000,99584000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
2,AAPL,3,2022-09-24,2022-09-24,USD,0000320193,2022-10-28,2022-10-27 18:01:14,2022,FY,...,-110749000000,0,-10952000000,24977000000,35929000000,122151000000,-10708000000,111443000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
3,AAPL,4,2021-09-25,2021-09-25,USD,0000320193,2021-10-29,2021-10-28 18:04:28,2021,FY,...,-93353000000,0,-3860000000,35929000000,39789000000,104038000000,-11085000000,92953000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
4,AAPL,5,2020-09-26,2020-09-26,USD,0000320193,2020-10-30,2020-10-29 18:06:25,2020,FY,...,-86820000000,0,-10435000000,39789000000,50224000000,80674000000,-7309000000,73365000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
5,MSFT,1,2024-06-30,2024-06-30,USD,0000789019,2024-07-30,2024-07-30 16:06:22,2024,FY,...,-37757000000,-210000000,-16389000000,18315000000,34704000000,118548000000,-44477000000,74071000000,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...
6,MSFT,2,2023-06-30,2023-06-30,USD,0000789019,2023-07-27,2023-07-27 16:01:56,2023,FY,...,-43935000000,-194000000,20773000000,34704000000,13931000000,87582000000,-28107000000,59475000000,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...
7,MSFT,3,2022-06-30,2022-06-30,USD,0000789019,2022-07-28,2022-07-28 16:06:19,2022,FY,...,-58876000000,-141000000,-293000000,13931000000,14224000000,89035000000,-23886000000,65149000000,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...
8,MSFT,4,2021-06-30,2021-06-30,USD,0000789019,2021-07-29,2021-07-29 16:21:55,2021,FY,...,-48486000000,-29000000,648000000,14224000000,13576000000,76740000000,-20622000000,56118000000,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...
9,MSFT,5,2020-06-30,2020-06-30,USD,0000789019,2020-07-30,2020-07-30 20:44:46,2020,FY,...,-46031000000,-201000000,2220000000,13576000000,11356000000,60675000000,-15441000000,45234000000,https://www.sec.gov/Archives/edgar/data/789019...,https://www.sec.gov/Archives/edgar/data/789019...


## ---------- VARIABILI IMPORTANTI ED ESEMPI ------------

### VARIABILI IMPORTANTI PER INCOME

Variabile | Descrizione | Frequenza reale di variazione

revenue | Ricavi totali della società | Trimestrale (earnings)

grossProfit | Ricavi - costo del venduto | Trimestrale

operatingIncome | Risultato operativo prima delle tasse e interessi | Trimestrale

netIncome | Utile netto | Trimestrale

eps (Earnings per Share) | Utile per azione | Trimestrale

interestExpense | Interessi passivi | Trimestrale

In [ ]:
# Esempio di output per income

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "revenue": 391035000000,
        "costOfRevenue": 210352000000,
        "grossProfit": 180683000000,
        "grossProfitRatio": 0.4620634982,
        "researchAndDevelopmentExpenses": 31370000000,
        "generalAndAdministrativeExpenses": 0,
        "sellingAndMarketingExpenses": 0,
        "sellingGeneralAndAdministrativeExpenses": 26097000000,
        "otherExpenses": 0,
        "operatingExpenses": 57467000000,
        "costAndExpenses": 267819000000,
        "interestIncome": 0,
        "interestExpense": 0,
        "depreciationAndAmortization": 11445000000,
        "ebitda": 134661000000,
        "ebitdaratio": 0.3443707085,
        "operatingIncome": 123216000000,
        "operatingIncomeRatio": 0.3151022287,
        "totalOtherIncomeExpensesNet": 269000000,
        "incomeBeforeTax": 123485000000,
        "incomeBeforeTaxRatio": 0.3157901467,
        "incomeTaxExpense": 29749000000,
        "netIncome": 93736000000,
        "netIncomeRatio": 0.2397125577,
        "eps": 6.11,
        "epsdiluted": 6.08,
        "weightedAverageShsOut": 15343783000,
        "weightedAverageShsOutDil": 15408095000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER BALANCE

Variabile | Descrizione | Frequenza reale

totalAssets | Totale attivo | Trimestrale

totalLiabilities | Totale passivo | Trimestrale

cashAndCashEquivalents | Cassa disponibile | Trimestrale

totalDebt | Totale debito (a breve e lungo termine) | Trimestrale

inventory | Scorte | Trimestrale

shareholderEquity | Capitale proprio | Trimestrale

In [ ]:
# Esempio di output per balance

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "cashAndCashEquivalents": 29943000000,
        "shortTermInvestments": 35228000000,
        "cashAndShortTermInvestments": 65171000000,
        "netReceivables": 66243000000,
        "inventory": 7286000000,
        "otherCurrentAssets": 14287000000,
        "totalCurrentAssets": 152987000000,
        "propertyPlantEquipmentNet": 45680000000,
        "goodwill": 0,
        "intangibleAssets": 0,
        "goodwillAndIntangibleAssets": 0,
        "longTermInvestments": 91479000000,
        "taxAssets": 19499000000,
        "otherNonCurrentAssets": 55335000000,
        "totalNonCurrentAssets": 211993000000,
        "otherAssets": 0,
        "totalAssets": 364980000000,
        "accountPayables": 68960000000,
        "shortTermDebt": 22511000000,
        "taxPayables": 26601000000,
        "deferredRevenue": 8249000000,
        "otherCurrentLiabilities": 50071000000,
        "totalCurrentLiabilities": 176392000000,
        "longTermDebt": 96548000000,
        "deferredRevenueNonCurrent": 0,
        "deferredTaxLiabilitiesNonCurrent": 0,
        "otherNonCurrentLiabilities": 35090000000,
        "totalNonCurrentLiabilities": 131638000000,
        "otherLiabilities": 0,
        "capitalLeaseObligations": 12430000000,
        "totalLiabilities": 308030000000,
        "preferredStock": 0,
        "commonStock": 83276000000,
        "retainedEarnings": -19154000000,
        "accumulatedOtherComprehensiveIncomeLoss": -7172000000,
        "othertotalStockholdersEquity": 0,
        "totalStockholdersEquity": 56950000000,
        "totalEquity": 56950000000,
        "totalLiabilitiesAndStockholdersEquity": 364980000000,
        "minorityInterest": 0,
        "totalLiabilitiesAndTotalEquity": 364980000000,
        "totalInvestments": 126707000000,
        "totalDebt": 106629000000,
        "netDebt": 76686000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER CASHFLOW

Variabile | Descrizione | Frequenza reale

operatingCashFlow | Flussi di cassa dalle operazioni | Trimestrale

capitalExpenditure | Investimenti in capitale (CAPEX) | Trimestrale

freeCashFlow | Cash flow operativo - CAPEX | Trimestrale

dividendsPaid | Dividendi distribuiti | Trimestrale

In [ ]:
# Esempio di output per cashflow
[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "netIncome": 93736000000,
        "depreciationAndAmortization": 11445000000,
        "deferredIncomeTax": 0,
        "stockBasedCompensation": 11688000000,
        "changeInWorkingCapital": 3651000000,
        "accountsReceivables": -5144000000,
        "inventory": -1046000000,
        "accountsPayables": 6020000000,
        "otherWorkingCapital": 3821000000,
        "otherNonCashItems": -2266000000,
        "netCashProvidedByOperatingActivities": 118254000000,
        "investmentsInPropertyPlantAndEquipment": -9447000000,
        "acquisitionsNet": 0,
        "purchasesOfInvestments": -48656000000,
        "salesMaturitiesOfInvestments": 62346000000,
        "otherInvestingActivites": -1308000000,
        "netCashUsedForInvestingActivites": 2935000000,
        "debtRepayment": -5998000000,
        "commonStockIssued": 0,
        "commonStockRepurchased": -94949000000,
        "dividendsPaid": -15234000000,
        "otherFinancingActivites": -5802000000,
        "netCashUsedProvidedByFinancingActivities": -121983000000,
        "effectOfForexChangesOnCash": 0,
        "netChangeInCash": -794000000,
        "cashAtEndOfPeriod": 29943000000,
        "cashAtBeginningOfPeriod": 30737000000,
        "operatingCashFlow": 118254000000,
        "capitalExpenditure": -9447000000,
        "freeCashFlow": 108807000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]